This Feature Engineering Notebook is a step-forward from the learnings of the notebook `Model_Training_RAPIDS.ipynb` to decrease the use of computational resources to train our model hassle-free.<br>
In this notebook, we try to optimize the data types of the arrays for the features, labels and groups to save the computational resources required during feature extraction and also model training.<br>


### Results:
1. By converting the features array(X) from float64 to float16, we brough down the memory usage from around `7 GB to 1.5 GB`.
2. By converting the labels array(y) from int64 to uint8 we, we brought down the memory usage from `1.43e-04 GB to 1.7875e-05 GB`.
3. By converting the labels array(y) from int64 to int16 we, we brought down the memory usage from `1.43e-04 GB to 3.575e-05 GB`.

`While converting one data type to another it was ensured that the precision of the numbers inside the arrays was not lost`

### Importing Libraries

In [ ]:
import os
import cv2
import pandas as pd
import cupy as cp  # CuPy for GPU-based NumPy operations
import numpy as np
import tensorflow as tf
import scipy
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#!pip install cupy --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 167.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Y
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cupy)


### Feature Extraction

**Define the paths for the images**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths
dataset_dir = "/content/drive/My Drive/Fabric Detection Project/textures 3"
categories = ['cotton', 'corduroy', 'denim', 'linin', 'wool']

**Canny Edge Detection**

In [ ]:
def extract_canny_edge_detection(image):
    """ image must be passes to the function in grayscale"""
    # Step 1: Enhance contrast (optional)
    equalized_image = cp.asarray(cv2.equalizeHist(cp.asnumpy(image)))

    # Step 2: Apply Gaussian Blur to reduce noise
    blurred_image = cp.asarray(cv2.GaussianBlur(cp.asnumpy(equalized_image), (3, 3), 1))

    # Step 3: Apply Canny Edge Detection with adjusted thresholds (convert back and forth)
    edges = cp.asarray(cv2.Canny(cp.asnumpy(blurred_image), 30, 30))

    return edges

**Gabor Filtering**

In [ ]:
def extract_gabor_filters(image):
    """ image must be in grayscale"""

    def build_kernels():
        # Parameters
        gabor_kernels = []
        angles = [0, cp.pi/4, cp.pi/2, 3*cp.pi/4]  # Use CuPy for angles
        ksize = 31  # Size of the filter
        sigma = 4.0  # Standard deviation of the Gaussian envelope
        lambd = 10.0  # Wavelength of the sinusoidal factor
        gamma = 0.5  # Spatial aspect ratio
        psi = 0  # Phase offset

        # Create Gabor kernels
        for theta in np.deg2rad([45, 135]):  # Convert degrees to radians
            kernel = cp.asarray(cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)) # Using Cupy array
            gabor_kernels.append(kernel)

        return gabor_kernels


    gabor_kernels = build_kernels()

    gabor_features = []

    for kernel in gabor_kernels:
        fimg = cp.asarray(cv2.filter2D(cp.asnumpy(image), cv2.CV_8UC3, cp.asnumpy(kernel)))
        gabor_features.append(fimg)

    gabor_features = cp.array(gabor_features).flatten()

    return gabor_features

**Local Binary Pattern**

In [ ]:
def extract_local_binary_pattern(image):

    # Parameters
    radius = 1
    n_points = 8 * radius


    lbp = local_binary_pattern(cp.asnumpy(image), n_points, radius, method="uniform")
    (hist, _) = cp.histogram(cp.asarray(lbp).ravel(), bins=cp.arange(0, n_points + 3),
                             range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    return hist

**Feature Extraction**

In [ ]:
# Function to extract features from an image
def extract_features(image):
    # Convert to grayscale using CuPy arrays
    gray = cp.asarray(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

    # Canny edge detection
    edges = extract_canny_edge_detection(gray)

    # Gabor Filter responses
    gabor_features=  extract_gabor_filters(gray)

    # Local Binary Patterns (LBP)
    hist = extract_local_binary_pattern(gray)

    # Combine features: edges, Gabor, and LBP
    features = cp.hstack([edges.flatten(), gabor_features, hist])
    features = cp.asnumpy(features)

    return features # Return features back as NumPy array for further processing

### Modified code for grouping of images and also including the original image with its augmentations. Also creating only 4 augmentations of each original image

In [ ]:
# Prepare dataset, labels, and groups
X = []
y = []
groups = []  # This will store the group IDs

# Image Augmentation using TensorFlow
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.4
)

group_id = 0  # Initialize group ID

for category in categories:
    path = os.path.join(dataset_dir, category)
    label = category

    for count, img_name in enumerate(os.listdir(path), start=1):

        img_path = os.path.join(path, img_name)
        image = cv2.imread(img_path)

        try:
            # Apply data augmentation and extract features
            image = cv2.resize(image, (128, 128))  # Resize to a fixed size

            # Extract features from the original image
            original_features = extract_features(image)
            X.append(original_features)
            y.append(label)
            groups.append(group_id)  # Assign the group ID to the original image

            # Prepare for augmentation
            image = np.expand_dims(image, axis=0)
            aug_iter = datagen.flow(image, batch_size=1)

            # Perform 4 augmentations per image
            for _ in range(4):
                aug_img = next(aug_iter)[0].astype(np.uint8)
                features = extract_features(aug_img)
                X.append(features)
                y.append(label)
                groups.append(group_id)  # Assign the same group ID to the augmentations

            # Increment group ID for the next image and its augmentations
            group_id += 1

        except Exception as e:
            print(f"{e} Error processing image: {img_path}, {img_name}")


print()

OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 Error processing image: /content/drive/My Drive/Fabric Detection Project/textures 3/corduroy/.ipynb_checkpoints, .ipynb_checkpoints



## Feature Engineering

**Converting X, y, groups from lists to ndarrays**

In [ ]:
# Checking the data type of X,y and groups
print(f"Type of X {type(X)}")
print(f"Type of y {type(y)}")
print(f"Type of groups {type(groups)}")

Type of X <class 'list'>
Type of y <class 'list'>
Type of groups <class 'list'>


In [ ]:
# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)
groups = np.array(groups)

In [ ]:
# Checking the data type of X,y and groups
print(f"Type of X {type(X)}")
print(f"Type of y {type(y)}")
print(f"Type of groups {type(groups)}")

Type of X <class 'numpy.ndarray'>
Type of y <class 'numpy.ndarray'>
Type of groups <class 'numpy.ndarray'>


**Shape of X, y, groups**

In [ ]:
# Checking the shape of the dataset and the labels
print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Groups shape: {groups.shape}")

Dataset shape: (17875, 49162)
Labels shape: (17875,)
Groups shape: (17875,)


### Inspecting the dtypes to save memory

In [ ]:
print(f"Dtype of X {X.dtype}")
print(f"Dtype of y {y.dtype}")
print(f"Dtype of groups {groups.dtype}")

Dtype of X float64
Dtype of y <U8
Dtype of groups int64


**Inspecting the dtype of X**

In [ ]:
print(f"Dtype of X {X.dtype}")

Dtype of X float64


In [ ]:
# Checking the size of X in GB
print(f"Size(GB) of X {X.nbytes/1e9}")

Size(GB) of X 7.030166


In [ ]:
import numpy as np

# Check if any values have non-zero decimals
has_decimals = np.any(X != np.floor(X))

if not has_decimals:
    # Convert the array to int32 if no decimals
    X = X.astype(np.int32)
    print("Array converted to int32 without decimals.")
else:
    print("Array contains non-zero decimals.")

print(f"Size(GB) of X {X.nbytes/1e9}")


Array contains non-zero decimals.
Size(GB) of X 7.030166


In [ ]:
len(np.unique(X))

5840

In [ ]:
# Check if values exceed float16 limits
float16_min = np.finfo(np.float16).min
float16_max = np.finfo(np.float16).max

# Check if any value is outside the float32 range
if np.any(X < float16_min) or np.any(X > float16_max):
    print("Array contains values outside the float16 range.")
else:
    print("All values are within the float16 range.")

All values are within the float16 range.


In [ ]:
# Converting X from float64 to float16
# Checking the size of X in GB
print(f"Size(GB) of X when float64 {X.nbytes/1e9}")

X = X.astype(np.float16)

# Checking the size of X in GB
print(f"Size(GB) of X after converting to float16 {X.nbytes/1e9}")

Size(GB) of X when float64 7.030166
Size(GB) of X after converting to float16 1.7575415


**Inspecting the dtype of y**

1. Encoding y

In [ ]:
# Example array with categories
categories = ['linen', 'cotton', 'wool', 'denim', 'corduroy']

# Create a dictionary for manual mapping
category_mapping = { 'corduroy': 1, 'cotton': 2, 'denim': 3, 'linin': 4, 'wool': 5}

# Convert to pandas Series (optional if already in pandas)
y_series = pd.Series(y)

# Map categories to numbers
mapped_categories = y_series.map(category_mapping)
y = np.array(mapped_categories)


print(y)
print(y.shape)

[2 2 2 ... 5 5 5]
(17875,)


In [ ]:
# Freeing up memory
del y_series
del mapped_categories

2. Checking the dtye of y

In [ ]:
print(f"Dtype of y {y.dtype}")

Dtype of y int64


In [ ]:
print(f"Size(GB) of y {y.nbytes/1e9}")

Size(GB) of y 0.000143


In [ ]:
print(np.unique(y))

[1 2 3 4 5]


In [ ]:
# Converting the dtype of y to uint8
y = y.astype(np.uint8)

print(f"Dtype of y {y.dtype}")
print(f"Size(GB) of y {y.nbytes/1e9}")

Dtype of y uint8
Size(GB) of y 1.7875e-05


**Inspecting the dtype of groups**

In [ ]:
print(f"Dtype of groups {groups.dtype}")

Dtype of groups int64


In [ ]:
# Get the min and max values for int8
int8_min = np.iinfo(np.int8).min
int8_max = np.iinfo(np.int8).max

# Check if all values fall within the int8 range
if np.all((groups >= int8_min) & (groups <= int8_max)):
    print("All values fall within the int8 range.")
else:
    print("Some values are outside the int8 range.")

Some values are outside the int8 range.


In [ ]:
# Get the min and max values for int16
int16_min = np.iinfo(np.int16).min
int16_max = np.iinfo(np.int16).max

# Check if all values fall within the int16 range
if np.all((groups >= int16_min) & (groups <= int16_max)):
    print("All values fall within the int16 range.")
else:
    print("Some values are outside the int16 range.")

All values fall within the int16 range.


In [ ]:
# Changing the dtype of groups into int32
print(f"Size of groups when int64 is {groups.nbytes/1e9}")
groups = groups.astype(np.int16)
print(f"Size of groups after changing to int16 is {groups.nbytes/1e9}")

Size of groups when int64 is 0.000143
Size of groups after changing to int16 is 3.575e-05


In [ ]:

# Path to the 'Extracted Features' directory
save_dir = "/content/drive/My Drive/Fabric Detection Project/Extracted Features"

# Create 'Extracted Features' directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save each array as a separate .npz file
np.savez(os.path.join(save_dir, 'X.npz'), data=X)
np.savez(os.path.join(save_dir, 'y.npz'), labels=y)
np.savez(os.path.join(save_dir, 'groups.npz'), groups=groups)

print(f"Data, labels, and groups have been saved to {save_dir}")


Data, labels, and groups have been saved to /content/drive/My Drive/Fabric Detection Project/Extracted Features


In [ ]:
# Estimate the size of each array in gigabytes (in memory)
import sys
data_size = sys.getsizeof(X) / (1024 * 1024 * 1024)
labels_size = sys.getsizeof(y) / (1024 * 1024 * 1024)
groups_size = sys.getsizeof(groups) / (1024 * 1024 * 1024)

print(f"Size of the data.npz :{data_size}")
print(f"Size of the labels.npz :{labels_size}")
print(f"Size of the groups.npz :{groups_size}")


Size of the data.npz :1.6368381939828396
Size of the labels.npz :1.6751699149608612e-05
Size of the groups.npz :3.339909017086029e-05


**Loading the saved data and labels**

In [ ]:
# Load from the .npz file
loaded_data = np.load('Extracted_features\\data.npz')
loaded_labels = np.load('Extracted_features\\labels.npz')
loaded_groups = np.load('Extracted_features\\groups.npz')

data_loaded = loaded_data["data"]
labels_loaded = loaded_labels["labels"]
groups_loaded = loaded_labels["groups"]

In [ ]:
# Checking the shape of the dataset and the labels
print(f"Dataset shape: {data_loaded.shape}")
print(f"Labels shape: {labels_loaded.shape}")
print(f"Groups shape: {groups_loaded.shape}")

Dataset shape: (17780, 49162)
Labels shape: (17780,)


From the above output we can see that we have **49162 features** and **17780 rows**